In [ ]:
import pandas as pd
import numpy as np
from emutools.tex import DummyTexDoc, StandardTexDoc
import arviz as az
from autumn.infrastructure.remote import springboard
from inputs.constants import SUPPLEMENT_PATH, RUNS_PATH, RUN_IDS, PRIMARY_ANALYSIS, BURN_IN
from aust_covid.calibration import get_targets
from emutools.calibration import plot_spaghetti, plot_param_hover_spaghetti
from aust_covid.plotting import plot_cdr_examples, plot_subvariant_props
from aust_covid.utils import add_image_to_doc
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'spaghetti', 'Example calibration runs', 'austcovid')
targets = get_targets(DummyTexDoc())
spaghetti = pd.read_hdf(RUNS_PATH / RUN_IDS['mob'] / 'output/results.hdf', 'spaghetti')

In [ ]:
key_indicators_fig = plot_spaghetti(spaghetti, ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number'], 2, targets)
key_outputs_title = 'Key outputs for randomly sampled runs from calibration algorithm.'
add_image_to_doc(key_indicators_fig.update_layout(showlegend=False), 'multioutput_spaghetti', 'svg', key_outputs_title, app_doc, 'Calibration results')
key_indicators_fig

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS['mob'] / 'output/calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[BURN_IN:])
sampled_idata = az.extract(burnt_idata, num_samples=10)
cdr_caption = 'Examples of the modelled effect of various starting CDR proportion parameters.'
print(cdr_caption)
cdr_fig = plot_cdr_examples(sampled_idata.variables['start_cdr'])
add_image_to_doc(cdr_fig, 'cdr_examples', 'svg', cdr_caption, app_doc, 'Model construction')
cdr_fig

In [ ]:
variant_prop_caption = 'Proportion of modelled cases attributable to each sub-variant over time. ' \
    'Solid curved lines, proportion of prevalence attributable to BA.1, ' \
    'Dashed curved lines, proportion of prevalence attributable to BA.1 or BA.2. ' \
    'Key dates for each variant are shown as vertical bars: blue, BA.1; red, BA.2; green, BA.5; ' \
    'dotted, first detection; dashed, \>1\%; solid, \>50\%. '
variant_prop_fig = plot_subvariant_props(spaghetti)
print(variant_prop_caption)
add_image_to_doc(variant_prop_fig, 'variant_prop_spaghetti', 'svg', 'Proportional prevalence of modelled sub-variants.', app_doc, 'Calibration results', caption=variant_prop_caption)
variant_prop_fig

In [ ]:
plot_param_hover_spaghetti(spaghetti['notifications_ma'], idata)

In [ ]:
app_doc.write_doc()